# Documentation

### Interior Point Method for SVM's

Consider the Support Vector Machine problem for binary classification:
$
\begin{align*}
\min_{w,b} \quad  &\frac{1}{m}\sum_{i=1}^m \max(1-y_i(w^Tx_i+b),0) +\lambda \|w\|_2^2 \tag{$P$}\\
\end{align*}
$

 We will write down the detailed algorithmic scheme to solve (P) through interior point method. We first transform problem into a QCQP:

$
\begin{align*}
\min_{w, b, \epsilon}\quad &  t\\
\text{s.t.} \quad & 
\lambda\|w\|_2^2 + \frac{1}{m}\sum_{i=1}^m \epsilon_i - t
\leq 0\\ &-y_i(w^\top x_i + b) + 1- \epsilon_i\leq 0, \; j=1,\ldots,m
\\&\epsilon_i\geq 0, \; j=1,\ldots,m
\end{align*}
$

We can express this optimization problem as a QCQP more concretely as:

$
\begin{align*}
\min_{y\in\mathbb{R}^{n+m+2}}\quad &  \bar{c}^\top y\\
\text{s.t.} \quad & q_0(y) \geq 0\\ &q_j(y)\geq 0, \; j=1,\ldots,m\\
&u_j(y) \geq 0, \; j=1,\ldots,m
\end{align*}
$

Where:
    
$
y = \begin{bmatrix}
w\\\epsilon\\b\\t\\ \end{bmatrix}\quad
\bar{c} = \begin{bmatrix}
\bar{0}^n\\\bar{0}^m\\0\\1
\
\end{bmatrix}
\quad
q_0 = -\lambda y^\top\Gamma_0y+ \gamma_0^\top y\quad q_j = \gamma_j^\top y -1,\quad j=1,\ldots,m
$

$
u_j = e_{i+n}^{{(n+m+2)}^\top} y,\quad
\Gamma_0 = \begin{bmatrix}
&I_{n\times n} &0\\ &0 &0
\end{bmatrix}
\quad
\gamma_0=
\begin{bmatrix}
\bar{0}^{n} \\-\frac{1}{m}\mathbb{1}\\
0\\1
\end{bmatrix}
\gamma_j=
\begin{bmatrix}
y_i x_i^\top \\e_i^m\\y_i\\0
\end{bmatrix}
$

Where $e_i^m$ is the unit basis vector with size $m$ and $i$'th element set to one and $\mathbb{1}$ is a column vector with all entries set to $1$.

We are now ready to solve this convex quadratic program using barrier interior point method with Newton steps. Note this scheme has two separate phases. First we move in the solution space to reach the quadratic convergence zone for Newton steps. This is where newton decrements are equal or less than $1/4$. Then phase 2 starts in which we  optimize the objective function. Denote:

$
F_\theta(y) = \min_y \theta \bar{c}^\top y - \sum_{j=0}^m \log(q_j) - \sum_{i=1}^m u_i = \min_y \theta \bar{c}^\top y + F(y)
$

Now:

$
\nabla q_0 = -2\lambda\Gamma_0 y + \gamma_0
\quad\nabla^2 q_0 = -2\lambda\Gamma_0
\quad\nabla q_j = \gamma_j
\quad\nabla u_j = e_{i+n}^{(n+m+2)}
$

And:

$
\nabla F(y) = -\left[
\sum_{j=0}^m \frac{\nabla q_j}{q_j} +  \sum_{j= 1}^m \frac{\nabla u_j}{u_j}
\right]
,\quad
\nabla^2 F(y) = \sum_{j=0}^m \frac{\nabla q_j\nabla^\top q_j}{q_j^2} + \sum_{j=1}^m \frac{\nabla u_j \nabla^\top u_j}{u_j^2}
-\frac{\nabla^2 q_0}{q_0}
$

Algorithm:
Start with $(y_0, \theta_0),\quad \theta_0 \geq 0$ and $\lambda_{F_{\theta_0}(y_0)} \leq 1/4$.

$
\theta_{k+1} = \theta_k(1 + \frac{\gamma}{\sqrt{2m + 1}})
$

$
y_{k+1}= y_k - [\nabla^2F(y_k)]^{-1}[\theta_{k+1} \bar{c} + \nabla F(y_k)]
$

If initial point is not available initialize IPM with damped Newton method and a feasible solution $y_0$ and $\theta_0 =1$
:

$
y_{k+1}= y_k - \gamma_f[\nabla^2F(y_k)]^{-1}[\theta_{0} \bar{c} + \nabla F(y_k)]
$

where $\gamma_f= \frac{1}{1+ \lambda_F}$ and $\lambda_F$ is the Newton decrement.

### Ellipsoid Method:

Ellipsoid method is generally slow specially if dimensionality of problem is big. For this dataset (32 features) Ellipsoid method performs reasonably well, although it is still subpar to interior point methods.
We first need to compute the first order oracle:

$
f(w, b) = \frac{1}{m}\sum_{i=1}^m \max(1-y_i(w^Tx_i+b),0) +\lambda \|w\|_2^2
=
\frac{1}{m}\sum_{i=1}^m g_i(w,b) +\lambda \|w\|_2^2
$

$
\nabla g_i(w,b) = \left\{ 
\begin{matrix}
&-y_i
\begin{bmatrix}
x_i\\ 1
\end{bmatrix}
&1 - y_i(w\top x_i +b)\geq0
\\&\bar{0}^{(n+1)} &O.W.
\end{matrix}
\right.
$

$
\nabla f(w, b) = \frac{1}{m}\sum_{i=1}^m \nabla g_i(w, b) + \begin{bmatrix}
2\lambda w\\ 0
\end{bmatrix}
$

Ellipsoid algorithm for SVM:
initialize with $c_0 = \begin{bmatrix}
w_0\\b_0
\end{bmatrix}$ and $Q_0 = R^2 I$. Denote $w = \nabla f(w, b)$

$
c_t = c_{t-1}- \frac{1}{n+1}\frac{Q_{t-1}w}{\sqrt{w^\top Q_{t-1}w}}
$

$
Q_t = \frac{n^2}{n^2 - 1}\left(
Q_{t-1} - \frac{2}{n+1}\frac{Q_{t-1} w w^\top Q_{t-1}}{w^\top Q_{t-1} w}
\right)
$

output best observed objective value.